In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
df = pd.read_csv('creditcard.csv')[:80_000]
df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [58]:
X = df.drop(columns=['Time', 'Amount', 'Class']).values
y = df['Class'].values
f"Shape of X = {X.shape} y = {y.shape}, #Fraud Cases = {y.sum()}"

'Shape of X = (80000, 28) y = (80000,), #Fraud Cases = 196'

In [59]:
from sklearn.linear_model import LogisticRegression

mod = LogisticRegression(class_weight={0:1, 1:2}, max_iter=1000)
mod.fit(X,y).predict(X).sum()

171

In [60]:
lr = LogisticRegression()
??lr.score

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, make_scorer

grid = GridSearchCV(
    estimator = LogisticRegression(max_iter=1000),
    param_grid={'class_weight': [{0:1, 1: v} for v in np.linspace(1,20,30)]},
    scoring={'precision': make_scorer(precision_score), 'recall': make_scorer(recall_score),
            'min_both': make_scorer(min_recall_precision)},
    refit='precision',
    return_train_score=True,
    cv = 10,
    n_jobs=-1)
grid.fit(X,y, sample_weight=np.log(1+df['Amount']))

In [ ]:
def min_recall_precision(est, y_true, y_pred):
    y_pred = est.predict(X)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    return min(recall, precision)

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y, grid.predict(X)), recall_score(y, grid.predict(X))

In [ ]:
pd.DataFrame(grid.cv_results_)

In [ ]:
plt.figure(figsize=(12,4))
df = pd.DataFrame(grid.cv_results_)
for score in ['mean_test_recall', 'mean_test_precision', 'mean_test_min_both']:
    plt.plot([_[1] for _ in df['param_class_weight']],
            df[score],
            label=score)
plt.legend()

In [ ]:
plt.figure(figsize=(12,4))
df = pd.DataFrame(grid.cv_results_)
for score in ['mean_test_recall', 'mean_train_precision']:
    plt.scatter(x=[_[1] for _ in df['param_class_weight']],
               y = df[score.replace('test', 'train')],
               label=score)
plt.legend()

In [ ]:
s = make_scorer(min_recall_precision)